In [1]:
import string
import re
import unicodedata
import bs4 as bs
import urllib.request

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gradio as gr
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Gerardo
[nltk_data]     Vilcamiza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Gerardo
[nltk_data]     Vilcamiza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Gerardo
[nltk_data]     Vilcamiza\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Gerardo
[nltk_data]     Vilcamiza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Obtención de los datos:

In [2]:
html_content = urllib.request.urlopen('https://es.wikipedia.org/wiki/Central_Restaurante')
html_content = html_content.read()

parse_html = bs.BeautifulSoup(html_content, 'lxml')
parrafos = parse_html.find_all('p')

full_text = ''.join(para.text for para in parrafos).lower()

## 2. Preprocesamiento:

In [3]:
def preprocess_clean_text(text):    
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    pattern = r'\[[0-9]*\]'
    text = re.sub(pattern, ' ', text)
    pattern = r'\s+'
    text = re.sub(pattern, ' ', text)
    #text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [4]:
text = preprocess_clean_text(full_text)

## 3. Dividir el texto en sentencias y en palabras:

In [5]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

print("Vocabulario:", len(words))

Vocabulario: 610


## 4. Funciones de ayuda para limpiar y procesar el input del usuario:

In [6]:
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

## 5. Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia:

In [7]:
def generate_response(user_input, corpus):
    response = ''
    corpus.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stopwords.words('spanish'))

    all_word_vectors = word_vectorizer.fit_transform(corpus)

    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Disculpe, no lo entiendo o no cuento con esa respuesta"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

## 6. Ensayar el sistema:

In [8]:
def bot_response(human_text):
    print("Q:", human_text)
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

C:\Users\Gerardo Vilcamiza\AppData\Local\Temp\ipykernel_980\3799335339.py:7: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Q: quién es el chef principal?
A: central restaurante es el restaurante principal del chef peruano virgilio martinez veliz y sirve como su taller en la investigacion y la integracion de insumos indigenas peruanos en la carta de comida del restaurante.
Q: dónde se ubica Central?
A: actualmente el restaurante se ubica en el distrito de barranco, en un espacio con un area aproximada de 3000 m2 que comparte con otras iniciativas del grupo como kjolle -el restaurante de la chef pia leon-, mayo bar y mater iniciativa, el centro de estudio biocultural del equipo de central.
Q: dame ejemplos de los insumos que usan
A: ejemplos de estos insumos son el cushuro, una cyanobacteria comestible de lagunas de altura; la arracacha, un tuberculo andino; y el paiche, un pez de agua dulce que vive en la cuenca del rio amazonas.
Q: quién es la esposa de Virgilio?
A: la chef de cocina es pia leon, esposa de martinez veliz.
Q: en qué años fue catalogado como el mejor de Latinoamérica?
A: ha sido catalogado c